Resources Used
- wget.download('https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/_downloads/da4babe668a8afb093cc7776d7e630f3/generate_tfrecord.py')
- Setup https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/install.html

# 0. Setup Paths

In [21]:
WORKSPACE_PATH = 'C:/Users/moon/Documents/posco_ai/codes/RealTimeObjectDetection/Tensorflow/'
SCRIPTS_PATH = WORKSPACE_PATH+'scripts'
APIMODEL_PATH = WORKSPACE_PATH+'models'
ANNOTATION_PATH = WORKSPACE_PATH+'workspace'+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'workspace'+'/images'
MODEL_PATH = WORKSPACE_PATH+'workspace'+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'workspace'+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'

In [22]:
# WORKSPACE_PATH = 'C:/Users/moon/Documents/posco_ai/codes/RealTimeObjectDetection/Tensorflow/workspace/'
# SCRIPTS_PATH = WORKSPACE_PATH + 'scripts'
# APIMODEL_PATH = WORKSPACE_PATH + 'models'
# ANNOTATION_PATH = WORKSPACE_PATH+'annotations'
# IMAGE_PATH = WORKSPACE_PATH+'images'
# MODEL_PATH = WORKSPACE_PATH+'models'
# PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'pre-trained-models'
# CONFIG_PATH = MODEL_PATH+'my_ssd_mobnet/pipeline.config'
# CHECKPOINT_PATH = MODEL_PATH+'my_ssd_mobnet/'

In [23]:
SCRIPTS_PATH

'C:/Users/moon/Documents/posco_ai/codes/RealTimeObjectDetection/Tensorflow/scripts'

In [24]:
PRETRAINED_MODEL_PATH

'C:/Users/moon/Documents/posco_ai/codes/RealTimeObjectDetection/Tensorflow/workspace/pre-trained-models'

# 1. Create Label Map

In [25]:
labels = [
    {'name':'question', 'id':1}, 
    {'name':'one', 'id':2},
    {'name':'two', 'id':3},
    {'name':'three', 'id':4},
    {'name':'four', 'id':5}
]

with open(ANNOTATION_PATH + '\label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 2. Create TF records

In [26]:
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x{IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}

Successfully created the TFRecord file: C:/Users/moon/Documents/posco_ai/codes/RealTimeObjectDetection/Tensorflow/workspace/annotations/train.record
Successfully created the TFRecord file: C:/Users/moon/Documents/posco_ai/codes/RealTimeObjectDetection/Tensorflow/workspace/annotations/test.record


# 3. Download TF Models Pretrained Models from Tensorflow Model Zoo

In [27]:
!cd Tensorflow && git clone https://github.com/tensorflow/models

fatal: destination path 'models' already exists and is not an empty directory.


In [28]:
# wget.download('http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz')
# !mv ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz {PRETRAINED_MODEL_PATH}
# !cd {PRETRAINED_MODEL_PATH} && tar -zxvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

# 4. Copy Model Config to Training Folder

In [29]:
PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'

'C:/Users/moon/Documents/posco_ai/codes/RealTimeObjectDetection/Tensorflow/workspace/pre-trained-models/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'

In [30]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 

In [31]:
!mkdir {'Tensorflow\workspace\models\\'+CUSTOM_MODEL_NAME}
# !cp {PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config'} {MODEL_PATH+'/'+CUSTOM_MODEL_NAME}
# 코드 실행되지않아 드래그앤 드랍으로 복사해주었음.

하위 디렉터리 또는 파일 Tensorflow\workspace\models\my_ssd_mobnet이(가) 이미 있습니다.


# 5. Update Config For Transfer Learning

In [32]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [33]:
CONFIG_PATH

'C:/Users/moon/Documents/posco_ai/codes/RealTimeObjectDetection/Tensorflow/workspace/modelsmy_ssd_mobnet/pipeline.config'

In [34]:
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'

In [35]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

In [36]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(CONFIG_PATH, "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [37]:
pipeline_config.model.ssd.num_classes = 2
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = PRETRAINED_MODEL_PATH+'/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0'
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/train.record']
pipeline_config.eval_input_reader[0].label_map_path = ANNOTATION_PATH + '/label_map.pbtxt'
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [ANNOTATION_PATH + '/test.record']

In [38]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(CONFIG_PATH, "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# 6. Train the model

In [39]:
print("""python {}/research/object_detection/model_main_tf2.py --model_dir={}/{} --pipeline_config_path={}/{}/pipeline.config --num_train_steps=10000""".format(APIMODEL_PATH, MODEL_PATH,CUSTOM_MODEL_NAME,MODEL_PATH,CUSTOM_MODEL_NAME))

python C:/Users/moon/Documents/posco_ai/codes/RealTimeObjectDetection/Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=C:/Users/moon/Documents/posco_ai/codes/RealTimeObjectDetection/Tensorflow/workspace/models/my_ssd_mobnet --pipeline_config_path=C:/Users/moon/Documents/posco_ai/codes/RealTimeObjectDetection/Tensorflow/workspace/models/my_ssd_mobnet/pipeline.config --num_train_steps=10000


# 7. Load Train Model From Checkpoint

In [50]:
# !pip install tf_slim

In [51]:
# !pip3 install official
# !conda install official

In [52]:
# !pip install tf-models-official

In [40]:
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [41]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

In [42]:
# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)

In [43]:
CHECKPOINT_PATH

'C:/Users/moon/Documents/posco_ai/codes/RealTimeObjectDetection/Tensorflow/workspace/models/my_ssd_mobnet/'

In [44]:
detection_model

In [45]:
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-3')).expect_partial()

In [46]:
@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# 8. Detect in Real-Time

In [47]:
import cv2 
import numpy as np

In [48]:
category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')

In [49]:
# cap.release()

In [50]:
# Setup capture
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [51]:
while True: 
    ret, frame = cap.read()
    image_np = np.array(frame)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.5,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        break

KeyboardInterrupt: 

In [73]:
detections = detect_fn(input_tensor)

In [74]:
detections

{'detection_boxes': <tf.Tensor: shape=(1, 100, 4), dtype=float32, numpy=
 array([[[0.00000000e+00, 4.38601971e-01, 9.75114107e-01, 1.00000000e+00],
         [2.67330289e-01, 5.59339285e-01, 9.55487132e-01, 1.00000000e+00],
         [0.00000000e+00, 4.33669120e-01, 6.49884641e-01, 9.89339471e-01],
         [0.00000000e+00, 4.49796081e-01, 1.00000000e+00, 1.00000000e+00],
         [0.00000000e+00, 4.49861735e-01, 7.16423631e-01, 9.96961594e-01],
         [0.00000000e+00, 3.21225107e-01, 9.24297512e-01, 1.00000000e+00],
         [5.83111346e-01, 4.59903479e-02, 1.00000000e+00, 4.84664440e-01],
         [7.46262968e-02, 8.17295313e-02, 8.33453536e-01, 1.00000000e+00],
         [4.16998744e-01, 5.29147148e-01, 1.00000000e+00, 1.00000000e+00],
         [5.53414762e-01, 6.90213919e-01, 9.88784254e-01, 1.00000000e+00],
         [5.84233820e-01, 0.00000000e+00, 9.75010693e-01, 1.99658021e-01],
         [0.00000000e+00, 2.47317851e-02, 8.31789851e-01, 4.23049062e-01],
         [3.53768408e-01, 5